In [39]:
# Pip Install If you do not have these packages
# Python version = 3.10
!pip install torch==2.0.1
!pip install numpy==1.24.3
!pip install scikit-learn==1.3.0
!pip install torch-geometric==2.3.1

You should consider upgrading via the '/Users/jasonzhao/PycharmProjects/blogcatalog/venv/bin/python/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/jasonzhao/PycharmProjects/blogcatalog/venv/bin/python/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/jasonzhao/PycharmProjects/blogcatalog/venv/bin/python/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/jasonzhao/PycharmProjects/blogcatalog/venv/bin/python/bin/python -m pip install --upgrade pip' command.


In [40]:
import torch
import numpy as np

# Load graph structure
graph_data = np.loadtxt('blogcatalogue-network.txt', skiprows=1, dtype=np.int64)
edge_index = torch.tensor(graph_data, dtype=torch.long).t().contiguous()

# Load node labels
labels_data = np.loadtxt('blogcatalogue-group.txt', skiprows=1, dtype=np.int64)
labels = torch.zeros((10312, 39))  # Assuming the number of nodes and classes from your provided data
for node_id, class_id in labels_data:
    labels[node_id, class_id] = 1  # It's a multi-label problem

In [41]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GNN(torch.nn.Module):
    def __init__(self):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(10312, 128)  # 10312 is the number of nodes (also the input dimension), 128 is an arbitrary hidden dimension
        self.conv2 = GCNConv(128, 128)
        self.conv3 = GCNConv(128, 128)
        self.out = torch.nn.Linear(128, 39)  # 39 is the number of classes

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        x = self.conv2(x, edge_index)
        x = F.relu(x)

        x = self.conv3(x, edge_index)
        x = F.relu(x)

        x = self.out(x)
        return torch.sigmoid(x)

In [ ]:
from torch_geometric.data import Data
from torch.optim import Adam
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Preparing the data
x = torch.eye(10312, dtype=torch.float)  # One-hot encoded input features
data = Data(x=x, edge_index=edge_index, y=labels)

# Split nodes for training and testing
train_mask, test_mask = train_test_split(np.arange(10312), test_size=0.5, random_state=42)
data.train_mask = torch.tensor(train_mask, dtype=torch.long)
data.test_mask = torch.tensor(test_mask, dtype=torch.long)
data = data.to(device)

# Setting up the model and optimizer
model = GNN().to(device)
optimizer = Adam(model.parameters(), lr=0.01)

# Training the model
model.train()
for epoch in range(400):  # For simplicity, we just loop 200 epochs
    optimizer.zero_grad()
    out = model(data)
    loss = F.binary_cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    if epoch % 50 == 0 and epoch > 0:
        print(f'{epoch} epochs has passed')

# Testing the model
model.eval()
with torch.no_grad():
    predictions = model(data)
    pred = predictions[data.test_mask].detach().cpu().numpy()
    true = data.y[data.test_mask].detach().cpu().numpy()


# Use the same prediction and true tensors as before
f1_scores = f1_score(true, pred > 0.5, average=None, zero_division=1)

print(f"F1 Score for each class: {f1_scores}")

cpu
50 epochs has passed
100 epochs has passed
150 epochs has passed
200 epochs has passed


In [ ]:
import matplotlib.pyplot as plt

# Assume f1_scores is the array of F1 scores calculated as before
classes = range(len(f1_scores))

plt.figure(figsize=(12, 6))  # Set the figure size
plt.bar(classes, f1_scores)  # Create a bar chart
plt.xlabel('Class')  # Set x-axis label
plt.ylabel('F1 Score')  # Set y-axis label
plt.title('F1 Score for Each Class')  # Set plot title
plt.xticks(classes)  # Set x-tick labels to be the classes
plt.ylim([0, 1])  # Optional: you might want to set y limits to [0, 1] as F1 Score is between 0 and 1
plt.grid(True)  # Optional: add a grid for better visualization
plt.show()